In [22]:
import pandas as pd
import torch 

- make three different function, all will return one emotion with confidence
- enable whisper and take urdu voice and do some prediction 
- make class which performs this all functions at once: 


# First model

In [1]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

In [54]:
def emo_det1(text):

    '''A Pretrained model from Hugging face, here It will return label and score or text'''
    tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
    model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
    predict1 = pipeline('sentiment-analysis', 
                        model='arpanghoshal/EmoRoBERTa')

    label  = predict1(text)[0]['label']
    score  = predict1(text)[0]['score']
    return label, score


In [55]:
l, s = emo_det1('I need help, kindly help me!')



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [56]:
print(l,s)

caring 0.9025200605392456


# Second Model

In [57]:
from transformers import pipeline

In [81]:
def emo_det2(text):
    pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)
    prediction = pipe(text)[0][0]['label'] # to give just top label
    # return label, score
    return prediction

emo_det2('I need help')


'sadness'

In [79]:
ls = [[{'label': 'sadness', 'score': 0.8441224098205566}]]
ls[0][0]['label']

'sadness'

In [61]:
ls = [{'label': 'anger', 'score': 0.07817116379737854}, {'label': 'disgust', 'score': 0.01308060809969902}, {'label': 'fear', 'score': 0.5388370156288147}, {'label': 'joy', 'score': 0.05621412396430969}, {'label': 'neutral', 'score': 0.22857415676116943}, {'label': 'sadness', 'score': 0.07132919877767563}, {'label': 'surprise', 'score': 0.013793745078146458}]

In [110]:
max(ls, key=lambda x:x['score'])

{'label': 'fear', 'score': 0.5388370156288147}

# Third Model


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

In [14]:

def emo_det3(text):
    
    tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
    model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

    input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

    output = model.generate(input_ids=input_ids,
                max_length=2)
    
    dec = [tokenizer.decode(ids) for ids in output]
    label = dec[0].split(' ')[1]
    return label    

emo_det3("i have a feeling i kinda lost my best friend") 

c:\Users\Abu Ubaida\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\modeling_auto.py:969: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


'sadness'

# **Video 2 Audio**

In [11]:
# first I need to extract audio from video
import moviepy.editor as me

def get_audio(path):
    video = me.VideoFileClip(path)
    audio = video.audio
    audio.write_audiofile('sample.mp3')
    print("Made MP3 file!")

# get_audio('./poetry.mp4')

# **Showing Audio**

In [12]:
import IPython
IPython.display.Audio("./content/sample.mp3")

# **Whisper**

In [3]:
import torch 
import torch.utils.checkpoint


In [8]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe("./content/sample.mp3", task='translate', fp16=False)
print(result["text"])

 My Lord has blessed me, how I have been worshiped, the reality is as if a pen has been drawn on me, the tongue of God has become. The address is, the Lord is the Lord of the man. You are the beauty of the garden, you are the color of spring. You are my body, you are the beauty of the art. I am the painter, you are my master. These mornings, these evenings, these days and nights, these colorful, beautiful, beautiful universes, the angels and the angels have made you the creator of the universe. You are the reference of my greatness, you are the light, you are the light. You have made the angels prostrate to you, that I have done for you whatever I have not done. Wherever this world is, these gatherings, these gatherings, these solitude, I have given you the light of the sky, I have given you the water to water your pitch, you have given me courage from the cities, I have given you the way in the mountains. The water, the air and the sun and the moon, the waves, the shore and the wind, 

- inside model.transcribe()

In [10]:
# import whisper

# model = whisper.load_model("base")

# # load audio and pad/trim it to fit 30 seconds
# audio = whisper.load_audio(r"./content/sample.mp3")
# audio = whisper.pad_or_trim(audio)

# # make log-Mel spectrogram and move to the same device as the model
# mel = whisper.log_mel_spectrogram(audio).to(model.device)

# # detect the spoken language
# _, probs = model.detect_language(mel)
# print(f"Detected language: {max(probs, key=probs.get)}")

# # decode the audio
# options = whisper.DecodingOptions(fp16=False)
# result = whisper.decode(model, mel, options)

# # print the recognized text
# print(result.text)

In [2]:
# function:
import whisper
def translate(audio):
    model = whisper.load_model("medium")
    result = model.transcribe("./content/sample.mp3", task='translate', fp16=False)
    return result['text']

# One Main Class

In [83]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from transformers import AutoTokenizer, AutoModelWithLMHead
import whisper
from scipy import stats
from collections import Counter


class audio2emo:
    def __init__(self, audio_path) -> None:
        self.audio = audio_path

    def translate(self, audio):
        model = whisper.load_model("medium")
        result = model.transcribe(audio, task='translate', fp16=False)
        return result['text']
    
    def emo_det1(self, text):
        '''A Pretrained model from Hugging face, here It will return label and score or text'''

        tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
        model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
        predict1 = pipeline('sentiment-analysis', 
                            model='arpanghoshal/EmoRoBERTa')

        label  = predict1(text)[0]['label']
        # score  = predict1(text)[0]['score']
        return label



    def emo_det2(self, text):
        pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)
        prediction = pipe(text)[0][0]['label'] # to give just top label
        # return label, score
        return prediction


    def emo_det3(self, text):
        
        tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
        model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

        input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

        output = model.generate(input_ids=input_ids,
                    max_length=2)
        
        dec = [tokenizer.decode(ids) for ids in output]
        label = dec[0]
        return label    

    def engine(self):
        text = self.translate(self.audio)
        L1 = self.emo_det1(text)
        L2 = self.emo_det2(text)
        L3 = self.emo_det3(text)
        _labels = [L1, L2, L3]

        # if there is mode then show it, else return full list
        ls = Counter(_labels).most_common()
        if ls[0][1] >1:
            return Counter(ls).most_common(1) # return most common
        else:
            return Counter(ls).most_common()

    
if __name__ == '__main__':
    audio_path = r'C:\Product\FS\Models\SENTIMENT-ANA\content\sample.mp3'
    obj = audio2emo(audio_path=audio_path)
    output = obj.engine()
    print(output)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
c:\Product\FS\Models\FS\lib\site-packages\transformers\models\auto\modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoMode

[(('admiration', 1), 1), (('neutral', 1), 1), (('<pad> joy', 1), 1)]


In [51]:
from scipy import stats
from collections import Counter

a = ['joy', 'angry', 'admiration','sad']
# stats.mode(a)[0][0]
# if there is mode then show it, else return full list
ls = Counter(a).most_common()
if ls[0][1] >1:
    print(Counter(a).most_common(1))
else:
    print(Counter(a).most_common())


[('joy', 1), ('angry', 1), ('admiration', 1), ('sad', 1)]


### **OPENAI**

In [7]:
import openai
response = openai.Image.create(
    prompt="a cat",
    n=1
    # size="256x256"
)
image_url = response['data'][0]['url']

InvalidRequestError: Billing hard limit has been reached